In [1]:
import numpy as np 
import pandas as pd
from datetime import timedelta
import datetime as dt
import matplotlib.pyplot as plt
import pickle as cPickle
from os.path import exists
from pathlib import Path
import csv
import datetime
import pytz

from pathlib import Path 
import time
import math

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold,cross_val_score
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
data = pd.read_csv('/content/drive/MyDrive/TNC_Collab_Results/training_data_all_20220203.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103356 entries, 0 to 103355
Columns: 300 entries, Unnamed: 0 to wb2sum
dtypes: float64(290), int64(5), object(5)
memory usage: 236.6+ MB


In [3]:
usgs_reg = pd.read_csv('/content/drive/MyDrive/TNC_Collab_Results/USGS_region_lookup.csv')
usgs_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   NA_L3CODE   11 non-null     object
 1   USGS_Group  11 non-null     object
dtypes: object(2)
memory usage: 304.0+ bytes


In [4]:
region_dict = dict(zip(usgs_reg.NA_L3CODE, usgs_reg.USGS_Group))
data['NewEco3'] = data['NewEco3'].replace(region_dict)
print(data['NewEco3'].unique())

['Xeric' 'Interior Mountains' 'North Coastal Mountains']


In [5]:
records_with_missing_data = len(data[data.isna().any(axis=1)])
print('Dropping {} records with missing data'.format(records_with_missing_data))
data = data.dropna(axis=0,how='any').copy()
#data.isnull().sum().sum()
data['flow_cfs'] = data['flow_cfs'] / data['DRAIN_SQKM']


Dropping 3 records with missing data


In [6]:
data.head(2)

,Unnamed: 0,gage_id,year,month,flow_cfs,comid,statistic,variable,ECO3,NHDPLUSREG,...,wb9,wb10,wb11,wb12,p6sum,p3sum,p2sum,wb6sum,wb3sum,wb2sum
0,0,10258000,1950,1,0.010533,22593497,mean,observed,8.0,18.0,...,40.085585,36.469271,13.544983,6.353392,173.076519,164.956173,145.791419,48.690715,22.244474,19.426220
1,1,10258500,1950,1,0.001901,22593537,mean,observed,8.0,18.0,...,9.309282,18.295394,20.328231,8.441266,83.834380,83.283639,71.869830,6.863135,4.810008,4.094633


In [7]:
data['year'].unique()

array([1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960,
       1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
       1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982,
       1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
       1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021])

In [8]:
data = data.loc[(data['year'] <=2015) & (data['NewEco3'] == 'Interior Mountains')] 
data

,Unnamed: 0,gage_id,year,month,flow_cfs,comid,statistic,variable,ECO3,NHDPLUSREG,...,wb9,wb10,wb11,wb12,p6sum,p3sum,p2sum,wb6sum,wb3sum,wb2sum
5,5,10281800,1950,1,0.072178,20267023,mean,observed,5.0,18.0,...,1.195967,1.141672,2.034422,3.674352,201.673616,190.623810,180.559667,163.108466,72.920863,51.780318
7,7,10310000,1950,1,0.155092,8924041,mean,observed,5.0,16.0,...,41.315219,3.801182,1.238397,2.476793,201.814896,174.104761,159.210717,197.704051,41.792674,29.085070
8,8,10340500,1950,1,0.099866,8933736,mean,observed,5.0,16.0,...,45.748842,7.542906,0.932284,1.864568,178.311970,152.456648,144.663358,131.915637,25.356233,17.289350
36,36,11213500,1950,1,0.130451,22050535,mean,observed,5.0,18.0,...,28.404976,25.169483,9.256541,5.561113,242.656564,226.828555,219.414842,265.816613,90.002620,64.057259
38,38,11230500,1950,1,0.115996,17118323,mean,observed,5.0,18.0,...,2.263617,1.911511,3.743803,7.487606,232.272943,212.707755,188.723944,401.225235,134.420207,85.632432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96572,70,11413320,2015,12,0.366928,8056721,mean,observed,5.0,18.0,...,81.044399,170.012356,146.084326,316.508613,195.360238,186.159157,178.273500,45.010424,17.979714,15.687527
96573,71,11413323,2015,12,0.275434,8056723,mean,observed,5.0,18.0,...,77.583794,162.821443,141.855573,309.292376,192.976314,183.828527,176.394915,37.094488,12.108754,9.985049
96574,72,11418000,2015,12,0.229454,8062589,mean,observed,5.0,18.0,...,72.789162,144.448443,109.917057,195.725166,217.106754,186.606580,181.680235,49.077665,23.308025,21.329948
96575,73,11427700,2015,12,0.773966,14992381,mean,observed,5.0,18.0,...,82.871711,157.911103,97.991877,137.806354,270.017022,220.061788,216.862745,82.963331,49.410935,47.032794


In [9]:
data['year'].unique()

array([1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960,
       1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
       1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982,
       1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
       1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015])

In [10]:
data['NewEco3'].unique()

array(['Interior Mountains'], dtype=object)

In [11]:
#dropping inessential columns
col = ['comid','Unnamed: 0','year','gage_id','statistic','variable','NHDPLUSREG','comidyear','ECO3','AggEcoRegion','NewEco3']
df = data.drop(columns = col)

In [12]:
df.head(2)

,month,flow_cfs,DRAIN_SQKM,SLOPE_PCT_30M,ELEV_MEAN_M_BASIN_30M,ELEV_MIN_M_BASIN_30M,ELEV_MAX_M_BASIN_30M,ElvRng,TOT_ECOL3_1,TOT_ECOL3_2,...,wb9,wb10,wb11,wb12,p6sum,p3sum,p2sum,wb6sum,wb3sum,wb2sum
5,1,0.072178,42.4575,44.33,2557.56,1162.12,4126.58,2964.46,0.0,0.0,...,1.195967,1.141672,2.034422,3.674352,201.673616,190.623810,180.559667,163.108466,72.920863,51.780318
7,1,0.155092,169.9299,27.69,2457.90,1750.21,3314.72,1564.51,0.0,0.0,...,41.315219,3.801182,1.238397,2.476793,201.814896,174.104761,159.210717,197.704051,41.792674,29.085070


In [13]:
df.columns

Index(['month', 'flow_cfs', 'DRAIN_SQKM', 'SLOPE_PCT_30M',
       'ELEV_MEAN_M_BASIN_30M', 'ELEV_MIN_M_BASIN_30M', 'ELEV_MAX_M_BASIN_30M',
       'ElvRng', 'TOT_ECOL3_1', 'TOT_ECOL3_2',
       ...
       'wb9', 'wb10', 'wb11', 'wb12', 'p6sum', 'p3sum', 'p2sum', 'wb6sum',
       'wb3sum', 'wb2sum'],
      dtype='object', length=289)

In [14]:
def x_train(month):
  train = df[df['month']==month]
  train.reset_index(drop=True,inplace=True)
  x_train = train.drop(columns = ['flow_cfs']).copy()
  return x_train

def y_train(month):
  train = df[df['month']==month]
  train.reset_index(drop=True,inplace=True)
  y_train = train.flow_cfs.values 
  return y_train

def x_valid(month):
  valid = df[df['month']==month]
  valid.reset_index(drop=True,inplace=True)
  x_valid = valid.drop(columns = ['flow_cfs']).copy()
  return x_valid

def y_valid(month):
  valid = df[df['month']==month]
  valid.reset_index(drop=True,inplace=True)
  y_valid = valid.flow_cfs.values
  return y_valid

def x_test(month):  
  test = df[df['month']==month]
  test.reset_index(drop=True,inplace=True)
  x_test = test.drop(columns = ['flow_cfs']).copy()
  return x_test

def y_test(month):
  test = df[df['month']==month]
  test.reset_index(drop=True,inplace=True)  
  y_test = test.flow_cfs.values
  return y_test

In [16]:
def hyperparameter_csv_append(month, model_run_date_time, r2, leaf, features, depth, estimator):
  if exists('/content/drive/MyDrive/TNC_Collab_Results/Results from USGS RF (Interior_M-all-comids) model/hyperparameter_'+str(month)+'.csv') == False:

    with open('/content/drive/MyDrive/TNC_Collab_Results/Results from USGS RF (Interior_M-all-comids) model/hyperparameter_'+str(month)+'.csv', 'w') as f:
      writer = csv.writer(f)
      header = ["model_run_date_time", "R2", "leaf", "features", "depth", "estimator"]
      writer.writerow(header)
      f.close()

  with open('/content/drive/MyDrive/TNC_Collab_Results/Results from USGS RF (Interior_M-all-comids) model/hyperparameter_'+str(month)+'.csv', 'a') as f:
    row = [str(model_run_date_time), str(r2), str(leaf), str(features), str(depth), str(estimator)]
    new_row = ",".join(row)
    f.write(new_row)
    f.write("\n")

# def get_random_values_hyperparameters():
#     min_samples_leaf = np.random.randint(low=50, high=100, size=1)[0]
#     max_features = np.random.uniform(0.2,0.5) #mtry in r default value (sqrt of total number of all predictors) #double check 17 express as?? (change dynamica base on month)
#     max_depth = np.random.randint(low=6, high=20, size=1)[0] #nodesize in r (1 for classification and 5 for regression)

def hyper_parameter_search(n_iter, month): 
 
    for i in range(n_iter):   
        # n_estimators = The number of trees in the forest. 
        estimators = [1000] #ntrees in r. Ted used 1000

        for count in estimators:
            rf = RandomForestRegressor(n_jobs = -1, n_estimators=count, min_samples_leaf=1,max_features="sqrt", max_depth=5, verbose=1) 
            # print("min_samples_leaf %d, max_features %.1f, max_depth %d, estimator %d" % (h1, h2, h3,count))
            # print ("min_samples_leaf %d, max_features %.1f, max_depth %d, estimator %d" % (1, "sqrt", 5,count))
            rf.fit(x_train(month), y_train(month))
            predictions = rf.predict(x_valid(month))
            features_sqrt = int(math.sqrt(len(x_train(month).columns)))

            # Mean absolute error (MAE)
            mae = mean_absolute_error(y_valid(month), predictions)
            # Mean squared error (MSE)
            mse = mean_squared_error(y_valid(month), predictions)
            # R-squared scores 
            r2 = r2_score(y_valid(month)+1, predictions+1)

            # Print metrics
            print('Mean Absolute Error:', round(mae, 2))
            print('Mean Squared Error:', round(mse, 2))
            print('R-squared scores:', round(r2, 2))            

            model_run_date_time = datetime.datetime.now(pytz.timezone('US/Pacific'))

            hyperparameter_csv_append(month, model_run_date_time, r2, 1, features_sqrt, 5, count)  


In [15]:
def train_final_model(month):

  df_usgs_reduced = pd.read_excel('/content/drive/MyDrive/TNC_Collab_Results/USGS_reduced_variables(Zimmerman_etal_2017_Supp_info2).xlsx', sheet_name = 1) 
  #Change the sheetname according to region, Sheet 1 = Interior_Mountains, Sheet 2 = Coastal_Mountains, Sheet 3 = Xeric
  df_usgs_reduced = df_usgs_reduced.loc[(df_usgs_reduced['Monthly.Stat'] == 'Mean') & (df_usgs_reduced['Month'] == month)] 
  reduced_var_list = df_usgs_reduced['Predictor'].unique().tolist()
  print("Number of variables for final model: {}".format(len(reduced_var_list)))
  print(reduced_var_list)

  dfp = pd.read_csv('/content/drive/MyDrive/TNC_Collab_Results/Results from USGS RF (Interior_M-all-comids) model/hyperparameter_'+str(month)+'.csv')
  dfp = dfp.sort_values(by="R2", ascending=False)
  
  best_leaf = dfp['leaf'].iloc[0]
  best_features = dfp['features'].iloc[0]
  best_depth = dfp['depth'].iloc[0]
  best_estimator = dfp['estimator'].iloc[0]

  reduced_columns_list = ['month']
  reduced_columns_list = reduced_columns_list + reduced_var_list

  # Train Reduced model
  rf = RandomForestRegressor(n_jobs = -1, min_samples_leaf=best_leaf,max_features=best_features, max_depth=best_depth, n_estimators=best_estimator,verbose=1) 
  rf.fit(x_train(month).loc[:, reduced_columns_list], y_train(month))
  predictions = rf.predict(x_test(month).loc[:, reduced_columns_list])
  r2 = r2_score(y_test(month)+1, predictions+1)
  model_run_date_time = datetime.datetime.now(pytz.timezone('US/Pacific'))
  print("R2 for USGS_Coastal Mountain_reduced model=",r2)

  rf_pickle_file = 'USGS_model_reduced_'+str(month)+'_r2_'+str(round(r2,4))+'_'+str(model_run_date_time)+'.pkl'
  with open('/content/drive/My Drive/TNC_Collab_Results/USGS_train_RF_with_reduced_features(Interior_M-all-comids)/'+rf_pickle_file, 'wb') as f:
    cPickle.dump(rf, f)

In [17]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [11]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 11


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.14
Mean Squared Error: 0.07
R-squared scores: 0.68


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.14
Mean Squared Error: 0.07
R-squared scores: 0.67


...

Mean Absolute Error: 0.14
Mean Squared Error: 0.07
R-squared scores: 0.66


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    7.5s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.5s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.5s finished


Mean Absolute Error: 0.14
Mean Squared Error: 0.07
R-squared scores: 0.68
Training final model for month 11
Number of variables for final model: 21
['wb0', 'p0', 'p3sum', 'p2sum', 'p1', 'p10', 't10', 'wb4', 't9', 'p6', 'wb10', 't11', 'p6sum', 'p5', 'wb5', 'p8', 'wb6', 'wb11', 'p2', 'p7', 'wb12']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   10.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_Coastal Mountain_reduced model= 0.8354814007356128


In [18]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [12]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 12


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.25
Mean Squared Error: 0.22
R-squared scores: 0.69


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.25
Mean Squared Error: 0.22
R-squared scores: 0.69


...

Mean Absolute Error: 0.25
Mean Squared Error: 0.21
R-squared scores: 0.69


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.25
Mean Squared Error: 0.21
R-squared scores: 0.69
Training final model for month 12
Number of variables for final model: 18
['p0', 'wb0', 'wb1', 'p2sum', 'p1', 'p3sum', 'p7', 'p6sum', 'RECHARGE', 'sedimentary', 'p10', 'p6', 'NO4AVE', 'p12', 'NO10AVE', 'WD_OCT_BASIN', 'wb2sum', 'HLR19']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    8.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_Coastal Mountain_reduced model= 0.888266328151547


In [19]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [1]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 1


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.28
Mean Squared Error: 0.23
R-squared scores: 0.75


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.28
Mean Squared Error: 0.24
R-squared scores: 0.74


...

Mean Absolute Error: 0.28
Mean Squared Error: 0.23
R-squared scores: 0.75


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.28
Mean Squared Error: 0.24
R-squared scores: 0.74
Training final model for month 1
Number of variables for final model: 21
['p0', 'wb0', 'wb1', 'wb2sum', 'p11', 'p6sum', 'p3sum', 'wb3sum', 'SILTAVE', 'sedimentary', 'p1', 'HLR19', 'p12', 'WD_OCT_BASIN', 'p2sum', 'wb8', 'p2', 'p9', 'PERDUN', 'PERHOR', 'SGEO44']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    8.6s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_Coastal Mountain_reduced model= 0.8899219219975589


In [20]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [2]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 2


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.28
Mean Squared Error: 0.24
R-squared scores: 0.68


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.28
Mean Squared Error: 0.24
R-squared scores: 0.68


...

Mean Absolute Error: 0.28
Mean Squared Error: 0.24
R-squared scores: 0.68


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.28
Mean Squared Error: 0.24
R-squared scores: 0.69
Training final model for month 2
Number of variables for final model: 20
['wb0', 'p0', 'wb1', 'wb2sum', 'wb3sum', 'PERHOR', 't5', 't1', 'HLR19', 'p4', 'RFACT', 'PERDUN', 'p11', 'WD_OCT_BASIN', 'SILTAVE', 'wb3', 'p2sum', 'p6sum', 'p9', 'PPTAVG_BASIN']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    8.5s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_Coastal Mountain_reduced model= 0.8364634005124966


In [21]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [3]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 3


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.27
Mean Squared Error: 0.17
R-squared scores: 0.72


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.27
Mean Squared Error: 0.17
R-squared scores: 0.72


...

Mean Absolute Error: 0.27
Mean Squared Error: 0.17
R-squared scores: 0.72


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.27
Mean Squared Error: 0.17
R-squared scores: 0.72
Training final model for month 3
Number of variables for final model: 15
['wb0', 'p0', 'wb3sum', 'wb2sum', 'p6sum', 'HLR19', 'wb1', 'PPTAVG_BASIN', 'PERHOR', 'RFACT', 't4', 'p4', 'PPTAVG_CAT', 'AnnMaxPrecip', 'p3sum']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   10.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_Coastal Mountain_reduced model= 0.8178358816635083


In [22]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [4]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 4


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.9s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.38
Mean Squared Error: 0.28
R-squared scores: 0.68


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.6s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.5s finished


Mean Absolute Error: 0.38
Mean Squared Error: 0.28
R-squared scores: 0.68


...

Mean Absolute Error: 0.38
Mean Squared Error: 0.28
R-squared scores: 0.68


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.38
Mean Squared Error: 0.28
R-squared scores: 0.68
Training final model for month 4
Number of variables for final model: 17
['RFACT', 'wb0', 'PPTAVG_BASIN', 'p6sum', 'pmpe', 'PPTAVG_CAT', 'AnnMaxPrecip', 't0', 'AnnMinPrecip', 'p3sum', 'RECHARGE', 'WD_NOV_BASIN', 'p0', 't1', 'p5', 'p1', 'p2sum']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    7.6s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_Coastal Mountain_reduced model= 0.7665576911982463


In [23]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [5]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 5


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.56
Mean Squared Error: 0.67
R-squared scores: 0.77


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.56
Mean Squared Error: 0.67
R-squared scores: 0.77


...

Mean Absolute Error: 0.56
Mean Squared Error: 0.68
R-squared scores: 0.77


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.9s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.56
Mean Squared Error: 0.67
R-squared scores: 0.77
Training final model for month 5
Number of variables for final model: 18
['wb0', 'p6sum', 'pmpe', 'WD_DEC_BASIN', 't1', 'RFACT', 'p3sum', 'NO200AVE', 'p4', 't2', 'CLAYAVE', 'T_MAX_BASIN', 't0', 'p5', 'WD_BASIN', 'WDMAX_BASIN', 'PPTAVG_CAT', 'WD_MAR_BASIN']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    7.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_Coastal Mountain_reduced model= 0.8435167129385996


In [24]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [6]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 6


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.5
Mean Squared Error: 0.63
R-squared scores: 0.77


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.5
Mean Squared Error: 0.63
R-squared scores: 0.77


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.5s finished


Mean Absolute Error: 0.51
Mean Squared Error: 0.64
R-squared scores: 0.77
Training final model for month 6
Number of variables for final model: 18
['p6sum', 'wb0', 't2', 't0', 'p3sum', 'WD_DEC_BASIN', 'p3', 't1', 'NO200AVE', 'p9', 'ELEV_MEAN_M_BASIN_30M', 'pmpe', 'CLAYAVE', 'p5', 'p2', 't3', 'p2sum', 'WDMAX_BASIN']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    9.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s


R2 for USGS_Coastal Mountain_reduced model= 0.8465699067825074


[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


In [25]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [7]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 7


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.26
Mean Squared Error: 0.19
R-squared scores: 0.71


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    4.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.26
Mean Squared Error: 0.19
R-squared scores: 0.7


...

Mean Absolute Error: 0.26
Mean Squared Error: 0.19
R-squared scores: 0.7


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.9s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.26
Mean Squared Error: 0.19
R-squared scores: 0.7
Training final model for month 7
Number of variables for final model: 16
['wb0', 't1', 'p6sum', 't3', 'p4', 'ELEV_MEAN_M_BASIN_30M', 't2', 'p6', 't0', 'p10', 'wb1', 'p7', 't8', 'p9', 'p3sum', 'p3']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    9.9s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_Coastal Mountain_reduced model= 0.7786326397290602


In [26]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [8]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 8


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.9s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.12
Mean Squared Error: 0.04
R-squared scores: 0.61


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.12
Mean Squared Error: 0.04
R-squared scores: 0.61


...

Mean Absolute Error: 0.12
Mean Squared Error: 0.04
R-squared scores: 0.61


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.9s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.12
Mean Squared Error: 0.04
R-squared scores: 0.61
Training final model for month 8
Number of variables for final model: 17
['wb0', 't2', 'wb1', 'SLOPE_PCT_30M', 't1', 't4', 't9', 'ELEV_MEAN_M_BASIN_30M', 'p11', 'HLR20', 'wb2sum', 'p6sum', 'p7', 'p5', 'wb10', 'p8', 'p4']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    9.3s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_Coastal Mountain_reduced model= 0.6024748227710224


In [27]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [9]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 9


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.09
Mean Squared Error: 0.04
R-squared scores: 0.59


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.9s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.09
Mean Squared Error: 0.04
R-squared scores: 0.59


...

Mean Absolute Error: 0.09
Mean Squared Error: 0.04
R-squared scores: 0.59


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.09
Mean Squared Error: 0.04
R-squared scores: 0.59
Training final model for month 9
Number of variables for final model: 22
['wb0', 'p0', 'wb1', 't0', 'NO200AVE', 'PPTAVG_BASIN', 'BFI_AVE', 'RFACT', 'pmpe', 't2', 'wb2sum', 'SLOPE_PCT_30M', 'wb3sum', 'LST32F_BASIN', 't3', 'p8', 'wb2', 't5', 't8', 'p6', 'p5', 'p6sum']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    8.6s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_Coastal Mountain_reduced model= 0.616843329479607


In [28]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [10]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 10


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.9s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.09
Mean Squared Error: 0.03
R-squared scores: 0.56


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.9s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.09
Mean Squared Error: 0.03
R-squared scores: 0.56


...

Mean Absolute Error: 0.09
Mean Squared Error: 0.03
R-squared scores: 0.56


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.9s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


Mean Absolute Error: 0.09
Mean Squared Error: 0.03
R-squared scores: 0.55
Training final model for month 10
Number of variables for final model: 22
['p0', 'wb0', 'wb6sum', 'BFI_AVE', 'WD_JUN_BASIN', 'PPTAVG_BASIN', 'Lperm', 'p8', 'p1', 'NO200AVE', 'CLAYAVE', 'p6sum', 'WD_BASIN', 'wb4', 'p2sum', 'wb3sum', 'p6', 'wb2', 'p12', 'wb1', 'wb2sum', 't10']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    8.6s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 1000 out of 1000 | elapsed:    0.3s finished


R2 for USGS_Coastal Mountain_reduced model= 0.5336774538160318
